In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import itertools

# Read 

In [2]:
colpo_meta = pd.read_excel(r"C:\Users\taral\OneDrive\Documents\Major project\Data\IARC\Colposcopic images\Colposcopic Cases Meta data.xlsx" )

colpo_meta.columns = colpo_meta.iloc[0]

colpo_meta = colpo_meta.iloc[1:].reset_index(drop=True)

colpo_meta = colpo_meta[~colpo_meta["Histopathology"].isin(("Not done", "Mucous polyp", "Tuberculosis of cervix with giant cells and granulation tissue"))]
colpo_meta = colpo_meta[["Case Number", "CaseID", "Histopathology"]]
colpo_meta.head(2)

,Case Number,CaseID,Histopathology
13,14,AAHE,Normal
15,16,AAIH,Normal


In [4]:
colpo_image_dat = pd.read_excel(r"C:\Users\taral\OneDrive\Documents\Major project\Data\IARC\Colposcopic images\Cases - Images.xlsx")
colpo_image_dat= colpo_image_dat[["Case Number", "File", "Type"]]
colpo_image_dat.head(2)

,Case Number,File,Type
0,1,AAAW1.jpg,After normal saline
1,1,AAAW3.jpg,After acetic acid


In [5]:
def find_matches(row, df2):
    matches = df2[df2['File'].str.startswith(row['CaseID'])]
    return matches

In [6]:
results = []

for _, row in colpo_meta.iterrows():
    matches = find_matches(row, colpo_image_dat)
    for _, match in matches.iterrows():
        print("FALSE:" + str(row['Case Number'] ) if row['Case Number'] != match['Case Number'] else "", end = "")
        results.append({
            'File': match['File'],
            'CaseID': row['CaseID'],
            'Case Number': row['Case Number'],
            'Type': match['Type'],
            'Class': row['Histopathology']
        })

final_colpo_set = pd.DataFrame(results)

In [7]:
final_colpo_set.head()

,File,CaseID,Case Number,Type,Class
0,AAHE1.jpg,AAHE,14,Speculum examination,Normal
1,AAHE0.jpg,AAHE,14,After normal saline with green filter,Normal
2,AAHE.jpg,AAHE,14,After acetic acid,Normal
3,AAHE3.jpg,AAHE,14,After acetic acid with higher magnification,Normal
4,AAHE4.jpg,AAHE,14,After Lugol’s iodine,Normal


In [8]:
final_colpo_set['Class'].value_counts()

Class
HSIL-CIN3                                                                                                                                      154
Normal                                                                                                                                          91
HSIL-CIN2                                                                                                                                       91
LSIL-CIN1                                                                                                                                       80
Invasive squamous cell carcinoma                                                                                                                39
Squamous cell carcinoma                                                                                                                         32
LSIL-HPV changes                                                                                                

# Move data

In [9]:
def copy_images(from_to):
    c = 0
    for fr, to in from_to:
        shutil.copy(fr, to)
        c +=1
        print(c, "done")

In [10]:
BASE_PATH_FROM = r"C:\Users\taral\OneDrive\Documents\Major project\Data\IARC\Colposcopic images"
BASE_PATH_TO = r"C:\Users\taral\OneDrive\Documents\Major project\Data\Compiled_All"

## Manage special cases first

In [11]:
CANCER_CLASSES  = [i for i in colpo_meta["Histopathology"].unique() if any(w in i.lower() for w in ["carcinoma", "cancer"])]
CANCER_CLASSES

['Cervical histopathology was invasive squamous cell cancer, vaginal histopathology was HSIL-VAIN 3, and vulvar histopathology was HSIL-VIN 3',
 'Microinvasive squamous cell cancer',
 'Invasive squamous cell carcinoma',
 'Microinvasive squamous cell carcinoma',
 'Squamous cell carcinoma',
 'Adenocarcinoma in situ',
 'Adenocarcinoma']

In [12]:
# generate image list for Cancer folder
cancer_path_to = BASE_PATH_TO + "\\Cancer\\"
cancer_from_to = []
for class_ in CANCER_CLASSES:
    rows = final_colpo_set[final_colpo_set['Class'] == class_]
    paths = [
        ( BASE_PATH_FROM + "\\Case {:03}\\{}".format(
            row["Case Number"],
            row["File"]
            ),
          cancer_path_to + row['File'])
            for _, row in rows.iterrows()]
    cancer_from_to.extend(paths)
cancer_from_to[:2]

[('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 140\\AACE0.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\Cancer\\AACE0.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 140\\AACE2.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\Cancer\\AACE2.jpg')]

In [13]:
copy_images(cancer_from_to)

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done


## Move rest of images

In [14]:
EXCLUDE = CANCER_CLASSES + ["HSIL-CIN3; LSIL-CIN1 in the anterior lip"]
CLASSES = set(final_colpo_set['Class'].unique()).difference(set(EXCLUDE))
CLASSES

{'HSIL-CIN2', 'HSIL-CIN3', 'LSIL-CIN1', 'LSIL-HPV changes', 'Normal'}

In [15]:
all_from_to = list()
for class_ in CLASSES:
    rows = final_colpo_set[final_colpo_set['Class'] == class_]
    paths = [
        ( BASE_PATH_FROM + "\\Case {:03}\\{}".format(
            row["Case Number"],
            row["File"]
            ),
          BASE_PATH_TO + "\\{}\\".format(class_) + row['File'])
            for _, row in rows.iterrows()]
    all_from_to.extend(paths)
all_from_to[:2]
    

[('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 106\\AAJI0.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\HSIL-CIN3\\AAJI0.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 106\\AAJI1.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\HSIL-CIN3\\AAJI1.jpg')]

In [16]:
copy_images(all_from_to)

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123 done
1

## Final Special case

In [17]:
SPECIAL = {"HSIL-CIN3; LSIL-CIN1 in the anterior lip":"HSIL-CIN3"}
special_from_to = list()
for label in SPECIAL.keys():
    rows = final_colpo_set[final_colpo_set['Class'] == label]
    paths = [
        ( BASE_PATH_FROM + "\\Case {:03}\\{}".format(
            row["Case Number"],
            row["File"]
            ),
          BASE_PATH_TO + "\\{}\\".format(SPECIAL[label]) + row['File'])
            for _, row in rows.iterrows()]
    special_from_to.extend(paths)
special_from_to[:]

[('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 107\\AABU0.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\HSIL-CIN3\\AABU0.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 107\\AABU1.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\HSIL-CIN3\\AABU1.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 107\\AABU2.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\HSIL-CIN3\\AABU2.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 107\\AABU4.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\Compiled_All\\HSIL-CIN3\\AABU4.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 107\\AABU6.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\

In [18]:
copy_images(special_from_to)

1 done
2 done
3 done
4 done
5 done
